In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

In [4]:
import pandas as pd
import awswrangler as wr
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')

Instalar PyAthena

In [5]:
!{sys.executable} -m pip install PyAthena

In [6]:
!{sys.executable} -m pip install geopandas

In [7]:
!{sys.executable} -m pip install mkl

In [8]:
!{sys.executable} -m pip install tensorflow --upgrade

Configurar PyAthena

In [9]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
from datetime import datetime
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [10]:
import geopandas as gpd
import glob
import os

In [11]:
import mkl

In [12]:
import tensorflow as tf
print(tf.__version__)

2.4.1


## Seleccionar paises

In [13]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [14]:
pais = 'BR'

## Asignar poligonos a usuarios

In [15]:
query = f'''SELECT iso_country_code,
                   caid, 
                   lat_mas_visitado as lat_final,
                   long_mas_visitado as long_final
            FROM historico_usuarios_deltas                            
            WHERE iso_country_code='{pais}'
            GROUP BY iso_country_code,
                     caid, 
                     lat_mas_visitado,
                     long_mas_visitado'''
df = cursor.execute(query).as_pandas()    
print(f"Tamaño tabla: {df.shape[0]}")
gdf_homes = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.long_final, df.lat_final), crs = 'epsg:4326')

Tamaño tabla: 143986741


In [16]:
gdf_homes

,iso_country_code,caid,lat_final,long_final,geometry
0,BR,086cac9dc16851a51578b9900ed634247ec6feba070054...,-3.076,-60.011,POINT (-60.01100 -3.07600)
1,BR,d49cf986a7c52612d7b9db6c42aa75edceb903a2c31c45...,-14.245,-46.470,POINT (-46.47000 -14.24500)
2,BR,f9f55fa9515fca687f79a2af273d591f618db5e76019b9...,-25.695,-48.487,POINT (-48.48700 -25.69500)
3,BR,af29c33b3f76779d24f0cff019bc3b53fda0abe57514c9...,-23.982,-48.877,POINT (-48.87700 -23.98200)
4,BR,78f4e83adf8b4cf7b09ab4def524d32f9de9f7a1fb2a76...,-23.595,-46.627,POINT (-46.62700 -23.59500)
...,...,...,...,...,...
143986736,BR,e878e74cace84f83dd3620677278601ceadf644629e953...,-29.831,-51.135,POINT (-51.13500 -29.83100)
143986737,BR,6a0f1f57902d2009f97c5e623596cf02c8f88b7f367190...,-5.699,-36.794,POINT (-36.79400 -5.69900)
143986738,BR,9d524a2aaa42da0b5b2f388db475bc343c20e295248565...,-16.640,-49.260,POINT (-49.26000 -16.64000)
143986739,BR,6be56b2339a6993332d839e39ebd42041ea67954087231...,-24.324,-50.636,POINT (-50.63600 -24.32400)


In [17]:
# Puntos de interes - Entidades bancarias
nom_shpefile = 'puntos_interes_CEF_Buffer' # Nombre del shapefile 
id_adm = 'id_agnc' # ID del punto de interes
loc_adm = f"Shapefiles/{pais}/{nom_shpefile}.shp"
path = glob.glob(f'{loc_adm}')
mapa = gpd.read_file(path[0], encoding = 'utf-8')
mapa['geometry'] = mapa['geometry'].to_crs(epsg = 4326)
gdf_homes.crs = mapa.crs
mapa = mapa.loc[:,[id_adm, 'geometry']]
mapa = mapa.rename(columns = {id_adm:'id_pi'})
gdf_homes_joined = gpd.sjoin(gdf_homes, mapa, op = 'within') 
gdf_homes_joined = gdf_homes_joined[['caid','iso_country_code', 'lat_final', 'long_final', 'id_pi']]
gdf_homes_joined['lat_final'] = round(gdf_homes_joined['lat_final'], 3)
gdf_homes_joined['long_final'] = round(gdf_homes_joined['long_final'], 3)
gdf_homes = gdf_homes_joined

In [18]:
gdf_homes

,caid,iso_country_code,lat_final,long_final,id_pi
917,594ccb4ff789dfacfe5f9e22f6b9e7721354a91f4e4928...,BR,-22.900,-43.178,9.999363e+09
203899,0f15e8da06a4705d39ea5b27ec75b2b662e479ee164501...,BR,-22.900,-43.178,9.999363e+09
233310,c93ce34c7ca7bb958b2afc98a5e2ae44977e648594c6e3...,BR,-22.900,-43.178,9.999363e+09
322126,fb109abac0bc5560da60e3eb93b2eabb1fade4ab8ad620...,BR,-22.900,-43.178,9.999363e+09
365774,968eb794459c8cff9af9d3cfb4d31c246c498f1d01aa74...,BR,-22.900,-43.178,9.999363e+09
...,...,...,...,...,...
115226360,2046d3610d334b979e63aa436310c1b04a370f2750e30a...,BR,-22.536,-44.765,9.999776e+09
131716331,8564a17cf3d38942cdac694617cddc49c0cfd7c7e0d7e5...,BR,-22.536,-44.765,9.999776e+09
132462868,52c690d07d2b20e3c13cfd55da714766c50604da0d411a...,BR,-22.536,-44.765,9.999776e+09
140924596,07f2bf8cbca8e5cee34a4db985e7401fe2502d095dfc67...,BR,-22.536,-44.765,9.999776e+09


In [19]:
table_pi = f'historico_usuarios_pi_{pais}'
wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df = gdf_homes,
    dataset = True,
    table = table_pi,
    database = "graphdata",
    path = f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/historico_usuarios_pi/{pais}/{table_pi}"
)

{'paths': ['s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/historico_usuarios_pi/BR/historico_usuarios_pi_BR/ffc6eb96e84540f2a46e9c1621e0e25f.snappy.parquet'],
 'partitions_values': {}}